과제: 이미지 크롤링

르누아르, 샤갈, 모딜리아니 등 유명 화가 작품 + 관련 데이터베이스 MongoDB로 만들어 올것. (이미지 태그로 이미지 저장, base64로 변환.)

데이터 200개.

In [195]:
import base64
from pymongo import MongoClient

In [196]:
client = MongoClient('localhost', 27017)

In [197]:
import requests
from selenium import webdriver
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException

In [98]:
driver = webdriver.Chrome()

In [219]:
def get_image(name):
    driver.get("https://artsandculture.google.com/?hl=ko")
    driver.find_element_by_css_selector(".sHnYxd > div").click()
    driver.implicitly_wait(3)
    driver.find_element_by_css_selector(".GmJoc").send_keys(name, "\n")
    driver.implicitly_wait(3)
    driver.find_element_by_css_selector(".FJEtJb > a").click()
    
    while True:
        try:
            driver.find_element_by_css_selector(".fe4pJf .bYeTje.CMCEae.BcYSHe").click()
        except WebDriverException as e:
            break
        except ElementNotVisibleException as e:
            break
            
    linksList = driver.find_elements_by_css_selector(".mqAyNe.p6VWJb.dm7raf.DYZP6e.uv3XYb.dOskXb a")
    images = client.get_database('images')
    collection = images.get_collection(name)
    for link in linksList:
        title = link.get_attribute('title')
        url = 'http://'+link.get_attribute('data-bgsrc')[2:]
        image = base64.encodebytes(requests.get(url).content)
        newitem = {
            'Title':title,
            'Image':image
        }
        collection.insert_one(newitem)

In [208]:
get_image("Vincent Van Gogh")

In [209]:
get_image("Gustav Klimt")

In [220]:
get_image("이중섭")

이중섭, Lee Jung Seob으로 함수 실행시 driver 작업은 정상적으로 실행되나 MongoDB collection 생성, 삽입 작업 안됨.